## Importing the csv file ans setting the index to collision_id

In [1]:
import pandas as pd
df = pd.read_csv('data/data_1000000.csv', index_col="collision_id")
df.head()

C:\Users\Guillaume\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
collision_id,,,,,,,,,,,,,,,,,,,,,
255278,2012-08-12T00:00:00.000,13:20,QUEENS,11373,40.736752,-73.877670,"(40.7367523, -73.8776702)",QUEENS BOULEVARD,BROADWAY,NaN,...,Unspecified,Unspecified,Unspecified,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN
38402,2012-09-04T00:00:00.000,19:15,MANHATTAN,10019,40.763658,-73.973464,"(40.7636578, -73.9734636)",WEST 58 STREET,5 AVENUE,NaN,...,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,TAXI,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
188040,2012-08-23T00:00:00.000,18:45,NaN,NaN,NaN,NaN,NaN,COURT STREET,WARREN STREET,NaN,...,Illness,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
294049,2012-09-06T00:00:00.000,0:45,STATEN ISLAND,10314,40.608054,-74.157353,"(40.6080537, -74.1573526)",MORANI STREET,VICTORY BOULEVARD,NaN,...,Unspecified,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
273074,2012-08-13T00:00:00.000,19:10,QUEENS,11377,40.753709,-73.902988,"(40.7537088, -73.9029876)",NORTHERN BOULEVARD,58 STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,NaN,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN


## Checking if the index is unique

In [2]:
df.index.is_unique

True

In [3]:
df.shape

(1000000, 28)

## Computing the percentage of missing values

In [4]:
df.isnull().sum()/df.shape[0] 

crash_date                       0.000000
crash_time                       0.000000
borough                          0.350498
zip_code                         0.350665
latitude                         0.078651
longitude                        0.078651
location                         0.078651
on_street_name                   0.231107
off_street_name                  0.480395
cross_street_name                0.780458
number_of_persons_injured        0.000016
number_of_persons_killed         0.000030
number_of_pedestrians_injured    0.000000
number_of_pedestrians_killed     0.000000
number_of_cyclist_injured        0.000000
number_of_cyclist_killed         0.000000
number_of_motorist_injured       0.000000
number_of_motorist_killed        0.000000
contributing_factor_vehicle_1    0.002928
contributing_factor_vehicle_2    0.151924
contributing_factor_vehicle_3    0.932037
contributing_factor_vehicle_4    0.985181
contributing_factor_vehicle_5    0.996037
vehicle_type_code1               0

## Removing columns with more than 40% of missing values

In [5]:
df = df.drop(df.loc[:,df.isnull().sum()/df.shape[0]  > 0.4], axis=1) 

## Removing blank spaces in all columns

In [6]:
# Remove leading and trailing withespaces
def strip_str(x):
    if type(x) is str:
        return x.strip()
    return x

In [7]:
df = df.applymap(strip_str)

## Replacing empty values of the factors of vehicules by Unspecified
### Moreover, separating the factors that are written in one column and separated with "/"

In [8]:
def factor_vehicule(x):
    
    if x["contributing_factor_vehicle_1"] != "nan":
        x["contributing_factor_vehicle_1"] = str(x["contributing_factor_vehicle_1"])
        if "/" in x["contributing_factor_vehicle_1"]:
            sp = x["contributing_factor_vehicle_1"].split("/")
            x["contributing_factor_vehicle_1"] = sp[0]
            x["contributing_factor_vehicle_2"] = sp[1]

    if x["contributing_factor_vehicle_1"] == "nan":
        x["contributing_factor_vehicle_1"] = "Unspecified"
        
    return x   

In [9]:
def factor_vehicule_2(x):
    
    if x["contributing_factor_vehicle_2"] != "nan":
        x["contributing_factor_vehicle_2"] = str(x["contributing_factor_vehicle_2"])
        if "/" in x["contributing_factor_vehicle_2"]:
            sp = x["contributing_factor_vehicle_2"].split("/")
            x["contributing_factor_vehicle_2"] = sp[1]

    if x["contributing_factor_vehicle_2"] == "nan":
        x["contributing_factor_vehicle_2"] = "Unspecified"
        
    return x     

In [10]:
df = df.apply(factor_vehicule, axis=1)
df = df.apply(factor_vehicule_2, axis=1) 

## Same as idea as below for separating with vehicule types

In [11]:
def type_vehicule(x):
    
    if x["vehicle_type_code1"] != x["vehicle_type_code1"] or x["contributing_factor_vehicle_2"] != "0":
        x["vehicle_type_code1"] = str(x["vehicle_type_code1"])
        if "/" in x["vehicle_type_code1"]:
            sp = x["vehicle_type_code1"].split("/")
            x["vehicle_type_code1"] = sp[0]
            x["vehicle_type_code2"] = sp[1]

    if x["vehicle_type_code1"] == "nan":
        x["vehicle_type_code1"] = "Unspecified"
        
    return x  

In [12]:
def type_vehicule2(x):
    
    if x["vehicle_type_code2"] != x["vehicle_type_code2"]:
        x["vehicle_type_code2"] = str(x["vehicle_type_code2"])
        if "/" in x["vehicle_type_code2"]:
            sp = x["vehicle_type_code2"].split("/")
            x["vehicle_type_code2"] = sp[1]

    if x["vehicle_type_code2"] == "nan":
        x["vehicle_type_code2"] = "Unspecified"
        
    return x  

## Consolidate main columns for vehicles

In [13]:
def uniform_vehicles(x):

    Ambulance = ["AMB", 'AMBUL', "Ambul", 'AMBU', 'ambul', 'AMBULANCE', 'AMBULACE', 'GEN  AMBUL', 'AMBULENCE',
                'FDNY AMBUL', 'Fdny ambul', 'White ambu', 'NYC AMBULA', 'AMBULENCE', 'abulance']
    Sedan = ["2 dr sedan", '4 dr sedan', "SMALL COM VEH(4 TIRES)","LARGE COM VEH(6 OR MORE TIRES)"]
    Van = ["van", 'Armored Truck', "VAN", 'Refrigerated Van']
    Taxi = ["TAXI"]
    Bus = ['School Bus', 'bus']
    Unspecified = ['UNKNO', 'Unkno', 'UNKNOWN', 'Unknown']
    Truck = ["TRUCK"]
    Other = ["OTHER"]
    Pickup = ["PICK-UP TRUCK"]
    Sta = ["STATION WAGON"]
    
    if x["vehicle_type_code1"] in Ambulance:
        x["vehicle_type_code1"] = "Ambulance"
    
    if x["vehicle_type_code1"] in Sedan:
        x["vehicle_type_code1"] = "Sedan"
        
    if x["vehicle_type_code1"] in Van:
        x["vehicle_type_code1"] = "Van" 
    
    if x["vehicle_type_code1"] in Taxi:
        x["vehicle_type_code1"] = "Taxi" 
        
    if x["vehicle_type_code1"] in Bus:
        x["vehicle_type_code1"] = "Bus"   
        
    if x["vehicle_type_code1"] in Unspecified:
        x["vehicle_type_code1"] = "Unspecified" 
        
    if x["vehicle_type_code1"] in Truck:
        x["vehicle_type_code1"] = "Truck"     
        
    if x["vehicle_type_code1"] in Other:
        x["vehicle_type_code1"] = "Other" 
        
    if x["vehicle_type_code1"] in Pickup:
        x["vehicle_type_code1"] = "Pick-up Truck" 
        
    if x["vehicle_type_code1"] in Sta:
        x["vehicle_type_code1"] = "Station Wagon" 

        
    return x

In [16]:
def uniform_vehicles_2(x):

    Ambulance = ["AMB", 'AMBUL', "Ambul", 'AMBU', 'ambul', 'AMBULANCE', 'AMBULACE', 'GEN  AMBUL', 'AMBULENCE',
                'FDNY AMBUL', 'Fdny ambul', 'White ambu', 'NYC AMBULA', 'AMBULENCE', 'abulance']
    Sedan = ["2 dr sedan", '4 dr sedan', "SMALL COM VEH(4 TIRES)", "LARGE COM VEH(6 OR MORE TIRES)"]
    Van = ["van", 'Armored Truck', "VAN", 'Refrigerated Van']
    Taxi = ["TAXI"]
    Bus = ['School Bus', 'bus', "BUS"]
    Unspecified = ['UNKNO', 'Unkno', 'UNKNOWN', 'Unknown']
    Truck = ["TRUCK"]
    Other = ["OTHER"]
    Pickup = ["PICK-UP TRUCK"]
    Sta = ["STATION WAGON"]
    Bike = ["BICYCLE"]
    
    if x["vehicle_type_code2"] in Ambulance:
        x["vehicle_type_code2"] = "Ambulance"
    
    if x["vehicle_type_code2"] in Sedan:
        x["vehicle_type_code2"] = "Sedan"
        
    if x["vehicle_type_code2"] in Van:
        x["vehicle_type_code2"] = "Van" 
    
    if x["vehicle_type_code2"] in Taxi:
        x["vehicle_type_code2"] = "Taxi" 
        
    if x["vehicle_type_code2"] in Bus:
        x["vehicle_type_code2"] = "Bus"   
        
    if x["vehicle_type_code2"] in Unspecified:
        x["vehicle_type_code2"] = "Unspecified" 
        
    if x["vehicle_type_code2"] in Truck:
        x["vehicle_type_code2"] = "Truck" 
        
    if x["vehicle_type_code2"] in Other:
        x["vehicle_type_code2"] = "Other" 
        
    if x["vehicle_type_code2"] in Pickup:
        x["vehicle_type_code2"] = "Pick-up Truck"     
     
    if x["vehicle_type_code2"] in Sta:
        x["vehicle_type_code2"] = "Station Wagon" 
        
    if x["vehicle_type_code2"] in Bike:
        x["vehicle_type_code2"] = "Bike"  
        
    return x

In [17]:
df = df.apply(type_vehicule, axis=1)
df = df.apply(type_vehicule2, axis=1)
df = df.apply(uniform_vehicles, axis=1)
df = df.apply(uniform_vehicles_2, axis=1)

In [45]:
df = df.applymap(strip_str)

## Only taking into account vehicle types that occur at least 100 times

In [47]:
a = df["vehicle_type_code1"].value_counts()
a = a[a < 1500].index

b = df["vehicle_type_code2"].value_counts()
b = b[b < 1500].index

df["vehicle_type_code1"] = df["vehicle_type_code1"].replace({df:'Other' for df in a})
df["vehicle_type_code2"] = df["vehicle_type_code2"].replace({df:'Other' for df in b})

In [44]:
df.loc[df["vehicle_type_code1_num"].isnull(),"vehicle_type_code1"]
#df["vehicle_type_code1_num"].unique()

collision_id
301303    SPORT UTILITY 
301256    SPORT UTILITY 
301753    SPORT UTILITY 
301415    SPORT UTILITY 
301556    SPORT UTILITY 
               ...      
107460    SPORT UTILITY 
107414    SPORT UTILITY 
106965    SPORT UTILITY 
106925    SPORT UTILITY 
107165    SPORT UTILITY 
Name: vehicle_type_code1, Length: 15267, dtype: object

## Assigning quantitative values to qualitative variables 

In [48]:
vehicle_1 = {
    "Sedan": 0,
    "Station Wagon": 1,
    "PASSENGER VEHICLE":2,
    "Taxi": 3,
    "Pick-up Truck": 4,
    "SPORT UTILITY": 5,
    "Box Truck": 6,
    "Bus": 7,
    "Other": 8,
    "Van": 9,
    "Unspecified": 10,
    "Bike": 11,
    "Tractor Truck Diesel": 12,
    "Motorcycle": 13,
    "Ambulance": 14,
    "Convertible":15,
    "Dump": 16,
    "Flat Bed": 17,
    "PK": 18,
    "Garbage or Refuse": 19
}
df["vehicle_type_code1_num"] = df["vehicle_type_code1"].apply(lambda x: vehicle_1.get(x))

In [49]:
vehicle_2 = {
    "Sport Utility Vehicle": 0,
    "Sedan": 1,
    "Unspecified": 2,
    "Station Wagon/Sport Utility Vehicle": 3,
    "Bike": 4,
    "Taxi": 5,
    "Bike": 6,
    "Other": 7,
    "Pick-up Truck": 8,
    "Box Truck": 9,
    "Bus": 10,
    "Tractor Truck Diesel": 11,
    "Motorcycle": 12,
    "Van":13,
    "E-Scooter": 14,
    "Dump": 15,
    "E-Bike": 16,
    "Flat Bed": 17,
    "STATION WAGON": 18,
    "SPORT UTILITY / STATION WAGON": 19,
    "PASSENGER VEHICLE":20,
}
df["vehicle_type_code2_num"] = df["vehicle_type_code2"].apply(lambda x: vehicle_2.get(x))

## Extracting date from crash date column

In [50]:
df['day']=df.apply(lambda x: str(x['crash_date'])[8:10],axis=1)
df['month']=df.apply(lambda x: str(x['crash_date'])[5:7],axis=1)
df['year']=df.apply(lambda x: str(x['crash_date'])[0:4],axis=1)

In [51]:
df['crash_date'] = pd.to_datetime(df[['year','month','day']])
df[['crash_date', 'day', 'month', 'year']].head()

,crash_date,day,month,year
collision_id,,,,
301815,2012-11-05,05,11,2012
3794971,2017-11-21,21,11,2017
4083808,2019-02-20,20,02,2019
3578022,2016-12-05,05,12,2016
3869029,2018-03-25,25,03,2018


## Removing rows with empty or zero latitude 

In [23]:
df = df[df['latitude'].notna()]
df = df[df['latitude'] != 0]
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,...,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,vehicle_type_code1_num,vehicle_type_code2_num,day,month,year
collision_id,,,,,,,,,,,,,,,,,,,,,
255278,2012-08-12,13:20,QUEENS,11373,40.736752,-73.877670,"(40.7367523, -73.8776702)",QUEENS BOULEVARD,3.0,0.0,...,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,2.0,20.0,12,08,2012
38402,2012-09-04,19:15,MANHATTAN,10019,40.763658,-73.973464,"(40.7636578, -73.9734636)",WEST 58 STREET,0.0,0.0,...,0,Driver Inattention,Distraction,Taxi,SPORT UTILITY / STATION WAGON,3.0,19.0,04,09,2012
294049,2012-09-06,0:45,STATEN ISLAND,10314,40.608054,-74.157353,"(40.6080537, -74.1573526)",MORANI STREET,2.0,0.0,...,0,Unspecified,Unspecified,PASSENGER VEHICLE,Unspecified,2.0,2.0,06,09,2012
273074,2012-08-13,19:10,QUEENS,11377,40.753709,-73.902988,"(40.7537088, -73.9029876)",NORTHERN BOULEVARD,0.0,0.0,...,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,2.0,20.0,13,08,2012
26715,2012-09-06,17:00,MANHATTAN,10017,40.753482,-73.980889,"(40.7534818, -73.9808885)",EAST 42 STREET,0.0,0.0,...,0,Unspecified,Unspecified,PASSENGER VEHICLE,Taxi,2.0,5.0,06,09,2012


## Filling borough values by assigning a value whenever, the location is in the range of the min-max longitude and latitude 

In [24]:

#Min-max Staten Island
min_long_SI = df[df["borough"] == "STATEN ISLAND"]['longitude'].min()
max_long_SI = df[df["borough"] == "STATEN ISLAND"]['longitude'].max()
min_lat_SI = df[df["borough"] == "STATEN ISLAND"]['latitude'].min()
max_lat_SI = df[df["borough"] == "STATEN ISLAND"]['latitude'].max()

#Min-max Bronx
min_long_Br = df[df['borough'] == "BRONX"]['longitude'].min()
max_long_Br = df[df['borough'] == "BRONX"]['longitude'].max()
min_lat_Br = df[df['borough'] == "BRONX"]['latitude'].min()
max_lat_Br = df[df['borough'] == "BRONX"]['latitude'].max()

#Min-max Queens
min_long_Q = df[df['borough'] == "QUEENS"]['longitude'].min()
max_long_Q = df[df['borough'] == "QUEENS"]['longitude'].max()
min_lat_Q = df[df['borough'] == "QUEENS"]['latitude'].min()
max_lat_Q = df[df['borough'] == "QUEENS"]['latitude'].max()

#Min-max BROOKLYN
min_long_B = df[df['borough'] == "BROOKLYN"]['longitude'].min()
max_long_B = df[df['borough'] == "BROOKLYN"]['longitude'].max()
min_lat_B = df[df['borough'] == "BROOKLYN"]['latitude'].min()
max_lat_B = df[df['borough'] == "BROOKLYN"]['latitude'].max()

#Min-max MANHATTAN
min_long_M = df[df['borough'] == "MANHATTAN"]['longitude'].min()
max_long_M = df[df['borough'] == "MANHATTAN"]['longitude'].max()
min_lat_M = df[df['borough'] == "MANHATTAN"]['latitude'].min()
max_lat_M = df[df['borough'] == "MANHATTAN"]['latitude'].max()

lis = []
for i in range(len(df["longitude"])):   
    
    if df.iloc[i,2] != df.iloc[i,2]:
        
        if (min_long_SI <= df.iloc[i,4] <= max_long_SI) & (min_lat_SI <= df.iloc[i,5] <= max_lat_SI):
            
            lis.append("STATEN ISLAND")
            
        elif (min_long_Br <= df.iloc[i,4] <= max_long_Br) & (min_lat_Br <= df.iloc[i,5] <= max_lat_Br):
            
            lis.append("BRONX")
            
        elif (min_long_Q <= df.iloc[i,4] <= max_long_Q) & (min_lat_Q <= df.iloc[i,5] <= max_lat_Q):
            
            lis.append("QUEENS")  
            
        elif (min_long_B <= df.iloc[i,4] <= max_long_B) & (min_lat_B <= df.iloc[i,5] <= max_lat_B):
            
            lis.append("BROOKLYN")  
            
        elif (min_long_M <= df.iloc[i,4] <= max_long_M) & (min_lat_M <= df.iloc[i,5] <= max_lat_M):
            
            lis.append("MANHATTAN")
            
        else: 
            lis.append(df.iloc[i,2])
    else: 
        lis.append(df.iloc[i,2])
        
df["borough"] = lis


## Filling empty values with their close locations (limit of 2). If there are still empty values, they take the value "Unspecified"

In [25]:
df.sort_values(by="location", inplace=True)
df["borough"].fillna(method ='ffill', limit = 2, inplace = True) 
df["zip_code"].fillna(method ='ffill', limit = 2, inplace = True) 

In [26]:
df["borough"].fillna("Unspecified", inplace = True)
df["zip_code"].fillna("Unspecified", inplace = True)
df["on_street_name"].fillna("Unspecified", inplace = True)

## Transform borough to quantitative values

In [27]:
df["borough"].value_counts()

BROOKLYN         245722
QUEENS           206570
MANHATTAN        165583
Unspecified      156728
BRONX            114538
STATEN ISLAND     30972
Name: borough, dtype: int64

In [28]:
def borough_quanti(x):
    
    if x["borough"] == "BROOKLYN":
        
        return 0
    
    if x["borough"] == "QUEENS":
        
        return 1
    
    if x["borough"] == "BRONX":
        
        return 2
    
    if x["borough"] == "MANHATTAN":
        
        return 3
    
    if x["borough"] == "STATEN ISLAND":
        
        return 4
    
    if x["borough"] == "Unspecified":
        
        return 5
    

In [29]:
df["borough_num"] = df.apply(borough_quanti, axis=1)

## Adding further column/information

#### Hour

In [30]:
df['hour']=df.apply(lambda x: x['crash_time'].split(":")[0],axis=1)

#### Day or night

In [31]:
def day_night(x):
    
    x["hour"] = int(x["hour"])
    if 7 <= x['hour'] <= 20:
        
        #day
        return 0
    
    else:
        
        #night
        return 1

In [32]:
df["day_night"] = df.apply(day_night, axis=1)

In [33]:
df["hour"].unique()

array(['11', '21', '12', '10', '17', '7', '0', '2', '14', '20', '16',
       '15', '19', '5', '18', '23', '9', '22', '4', '1', '8', '13', '6',
       '3'], dtype=object)

#### Victim(s)

In [34]:
df['victim']=df.apply(lambda x: x['number_of_persons_injured']+ 
                              x['number_of_persons_killed'],axis=1)

#### Season

In [35]:
def season(x):
    
    x["month"] = int(x["month"])
    x["day"] = int(x["day"])               
    if (x["month"] == 1) or (x["month"] == 2) or (x["month"] == 12 and x["day"] >= 21) or (x["month"] == 3 and x["day"] <= 20):
        
        #Winter
        return 0
                   
    elif (x["month"] == 4) | (x["month"] == 5) or (x["month"] == 3 and x["day"] >= 21) or (x["month"] == 6 and x["day"] <= 20):
        
        #Spring
        return 1
                   
    elif (x["month"] == 7) | (x["month"] == 8) or (x["month"] == 6 and x["day"] >= 21) or (x["month"] == 9 and x["day"] <= 20):
        
        #Summer
        return 2
                   
    elif (x["month"] == 10) | (x["month"] == 11) or (x["month"] == 9 and x["day"] >= 21) or (x["month"] == 12 and x["day"] <= 20):
        
        #Fall
        return "3"

In [36]:
df["season"] = df.apply(season, axis = 1)

## Checking and changing types of columns

In [37]:
df.dtypes

crash_date                       datetime64[ns]
crash_time                               object
borough                                  object
zip_code                                 object
latitude                                float64
longitude                               float64
location                                 object
on_street_name                           object
number_of_persons_injured               float64
number_of_persons_killed                float64
number_of_pedestrians_injured             int64
number_of_pedestrians_killed              int64
number_of_cyclist_injured                 int64
number_of_cyclist_killed                  int64
number_of_motorist_injured                int64
number_of_motorist_killed                 int64
contributing_factor_vehicle_1            object
contributing_factor_vehicle_2            object
vehicle_type_code1                       object
vehicle_type_code2                       object
vehicle_type_code1_num                  

In [53]:
convert_dict = {'day': int,
                'month': int,
                'year': int,
                "vehicle_type_code1_num": int,
                "vehicle_type_code2_num": int,
                "hour": int,
                "season": int,
                "day_night": int
               } 
  
df = df.astype(convert_dict)

In [54]:
df.to_csv("data/output_data_1000000.csv")